In [1]:
from pyspark.sql.functions import col

taxi_daily = spark.read.table("gold_taxi_daily")
fx_daily = spark.read.table("silver_fx_daily")

taxi_fx = (
    taxi_daily.alias("td")
    .join(
        fx_daily.alias("fd"),
        col("td.trip_date") == col("fd.fx_date"),
        "left"
    )
    .select(
        col("td.trip_date").alias("date"),
        col("td.trip_count"),
        col("td.total_fare_usd"),
        col("td.avg_fare_usd"),
        col("td.avg_trip_distance"),
        col("fd.usd_eur_rate"),
        (col("td.total_fare_usd") * col("fd.usd_eur_rate")).alias("total_fare_eur"),
        (col("td.avg_fare_usd") * col("fd.usd_eur_rate")).alias("avg_fare_eur")
    )
)

StatementMeta(, c5c12333-85d3-48f7-ba33-5a4623d2c7ef, 3, Finished, Available, Finished)

In [2]:
taxi_fx.show(20)

StatementMeta(, c5c12333-85d3-48f7-ba33-5a4623d2c7ef, 4, Finished, Available, Finished)

+----------+----------+------------------+------------------+------------------+------------+------------------+------------------+
|      date|trip_count|    total_fare_usd|      avg_fare_usd| avg_trip_distance|usd_eur_rate|    total_fare_eur|      avg_fare_eur|
+----------+----------+------------------+------------------+------------------+------------+------------------+------------------+
|2024-01-14|     91079|1655228.2599999898|18.173544505319445|3.3881434798361476|        NULL|              NULL|              NULL|
|2024-01-18|    106989|1946949.5800000113| 18.19766125489547|  3.07531306956788|      1.0875| 2117307.668250012| 19.78995661469882|
|2024-01-19|     93065|1631956.3300000052| 17.53566141943808|3.0692027077848762|      1.0887|1776710.8564710056| 19.09107458734224|
|2024-01-13|    101605|1788508.6000000164|17.602564834407918|3.8481721371979924|        NULL|              NULL|              NULL|
|2024-01-22|     83070|1572361.0999999885|18.928146141807975| 3.458928734802

In [3]:
(
    taxi_fx.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("gold_taxi_daily_eur")
)


StatementMeta(, c5c12333-85d3-48f7-ba33-5a4623d2c7ef, 5, Finished, Available, Finished)